In [264]:
# imports
import os
import json
import re

import torch 
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import numpy as np

from MDP import MDP

import stable_baselines3
import sb3_contrib

import gym

In [265]:
# check torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3080'

In [266]:
# create Neural Network

class Net(nn.Module):
    """
    input : 2 X 4 X 4 grid
    label : Move [0;6]
    """
    def __init__(self):
        super(Net, self).__init__()
        # first layer: input
        self.conv1 = nn.Conv2d(2, 32, 2, padding = 1)

        #second layer : 2nd convolution
        self.conv2 = nn.Conv2d(32, 64, 3, padding= 1)

        self.conv3 = nn.Conv2d(64, 128, 2)

        self.conv4 = nn.Conv2d(128, 64, 3)

        self.fc1 = nn.Linear(64, 32)

        self.fc2 = nn.Linear(32, 16)

        self.out = nn.Linear(16, 6)


    def forward(self, x):
        x = x.float()

        x = F.relu(self.conv1(x))

        x = F.relu(self.conv2(x))

        x = F.relu(self.conv3(x))

        x = F.relu(self.conv4(x))

        x = F.max_pool2d(x, kernel_size = 2, stride = 2)

        x = torch.flatten(x,start_dim=1)

        x = F.relu(self.fc1(x))

        x = F.relu(self.fc2(x))

        x = self.out(x)
    
        return x      

In [267]:
#creating model
net = Net()
net.cuda()
print(net)

params = list(net.parameters())
print(f"number of parameters: {len(params)}")

#loss function
loss = nn.CrossEntropyLoss()

#optimizer
optimizer = torch.optim.Adam(net.parameters())
optimizer

Net(
  (conv1): Conv2d(2, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=6, bias=True)
)
number of parameters: 14


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

In [268]:
net.load_state_dict(torch.load("Net"))

<All keys matched successfully>

In [269]:
#custom environment
from gym import spaces

class Gridworld(gym.Env):

    metadata = {"render.modes" : ["human"]}

    def __init__(self, dir = ["data", "data_easy", "data_medium"], type = ["train"], lambda1 = 0.01, lambda2 = 0.1, lambda3 = 1, load_optimal = False, epsilon = 0.1) -> None:
        super(Gridworld, self).__init__()
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(low = 0, high = 10, shape = (2, 4, 4))

        #available MDPs
        self.dir = dir
        self.type = type
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

        self.actions = ["move", "turnLeft", "turnRight", "pickMarker", "putMarker","finish"]
        self.actionsDict = {"move" : 0, "turnLeft" : 1, "turnRight" : 2, "pickMarker" : 3, "putMarker" : 4, "finish" : 5}

        self.epsilon = epsilon

        self.load_optimal = load_optimal

    def reset(self):
        nextDir, nextType = np.random.choice(self.dir), np.random.choice(self.type)
        self.nexti = np.random.choice(os.listdir(os.sep.join(["datasets", nextDir, nextType, "task"])))
        self.nexti = re.sub(r"\D", "", self.nexti)
        self.currentMDP = MDP(nextDir, nextType, self.nexti, lambda1= self.lambda1, lambda2 = self.lambda2, lambda3 =self.lambda3)

        # load optimal sequence if possible
        try:
            with open(os.sep.join(["datasets", nextDir, nextType, "seq", self.nexti + "_seq.json"])) as grid:
                grid = json.load(grid)
                self.optimal_seq = grid["sequence"]
                # current step
                assert self.optimal_seq[-1] == "finish"
        except:  
            self.optimal_seq = []
        self.steps = 0

        #with probability 1 - epsilon use either the optimal sequence or better masks
        self.use_optimal =  (np.random.rand() < (1 - self.epsilon)) and self.load_optimal

        return self.currentMDP.get_current_state()/10

    def step(self, action):
        nextState, rew, done, info = self.currentMDP.sample_next_state_and_reward(self.actions[action])
        self.steps += 1
        if self.steps > 500:
            return nextState, -self.lambda3, True, info 

        return nextState/10, rew - 0.01, done, info 
        
    def render(self):
        self.currentMDP.print_grid()

    def close(self):
        pass
    
    def action_masks(self):

        mat = self.currentMDP.get_current_state()
        if np.array_equal(mat[0], mat[1]):
            return np.array([0,0,0,0,0,1])

        # force agent to take optimal action if possible
        mask = self.currentMDP.action_mask()

        if self.use_optimal and self.optimal_seq:
            mask = np.zeros(6)
            mask[self.actionsDict[self.optimal_seq[self.steps]]] = 1
            return mask
        
        return mask

    # functions bellow are only used for inheritance 
    def get_MDP(self):
        return self.currentMDP

    def get_MDP_name(self):
        return self.nextDir, self.nextType, self.nexti

In [270]:
testEnv = Gridworld(dir = ["generated_med"], load_optimal= True)

In [271]:
testEnv.reset()
testEnv.render()
print(testEnv.action_masks())
_, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()
testEnv.action_masks()
_, r, _, _ = testEnv.step(np.argmax(testEnv.action_masks()))
print(r)
testEnv.render()

[[['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '>' '.']
  ['.' '.' '.' '.']]

 [['.' '.' '<' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']]]
[1. 1. 1. 0. 0. 0.]
-0.02
[[['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '>']
  ['.' '.' '.' '.']]

 [['.' '.' '<' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']]]
-0.02
[[['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '^']
  ['.' '.' '.' '.']]

 [['.' '.' '<' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']]]


In [272]:
np.random.seed(123)

def test_RL_models(model):
    for dir, num in zip(["data_easy", "data_medium", "data"], [80, 24, 480]):
        print("current data: " + dir)
        total_steps = 0
        valDataset = Gridworld(dir = [dir], type = ["val"], lambda1=0, lambda2=0, load_optimal= False)
        correct, total, optimal = 0, num*10, 0
        for task in range(int(total)):
            if task % num == num-1:
                print(f"{(task+1) / total *100} %, running acc: {(correct*100)/(task+1)}, task solved optimaly: {optimal*100/task} %, average steps to solve: {total_steps / task}")
            currMDP = valDataset.reset()
            lenOptimalSeq = len(valDataset.optimal_seq)
            done = False
            steps = 0
            while not done and steps < 50:
                action = model.predict(currMDP, action_masks = valDataset.action_masks())[0]
                currMDP, rew, done, _ = valDataset.step(action)
                steps += 1
                if rew > 0:
                    correct += 1
                    total_steps += steps
                    if steps == lenOptimalSeq:
                        optimal += 1

                
        

In [273]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env

import torch.nn.functional as F
from torch import nn

class CustomFeatureExtractorTorch(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 16):
        super().__init__(observation_space, features_dim)

        self.conv1 = net.conv1
        self.conv2 = net.conv2
        self.conv3 = net.conv3
        self.conv4 = net.conv4
        
        self.fc1 = net.fc1
        self.fc2 = net.fc2
  
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x, start_dim=1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

net_arch = [
    64, 128,
    dict(vf = [64, 32, 18], pi = [32, 16, 6])
]

policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [274]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.env_util import make_vec_env
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy

In [275]:
policy_kwargs = dict(
    features_extractor_class = CustomFeatureExtractorTorch,
    net_arch = net_arch
)

In [276]:
FinalEnv = make_vec_env(Gridworld, n_envs= 2  , 
    env_kwargs={ "lambda1" : 0.1, "lambda2" : 0.5, "lambda3": 1, 
            "dir" : ["data_easy","data_medium", "data", "generated_imitation", "generated_easy", "generated_med"], "load_optimal" : True, "epsilon" : 0.2})

FinalModel = MaskablePPO(MaskableActorCriticPolicy, FinalEnv, policy_kwargs= policy_kwargs,  verbose = 1, n_steps= 500)

FinalModel.learn(2.5 * 1e5)

Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.9     |
|    ep_rew_mean     | -0.812   |
| time/              |          |
|    fps             | 196      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 1000     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 36.3          |
|    ep_rew_mean          | -2.2          |
| time/                   |               |
|    fps                  | 247           |
|    iterations           | 2             |
|    time_elapsed         | 8             |
|    total_timesteps      | 2000          |
| train/                  |               |
|    approx_kl            | 0.00021964176 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.83         |
|    explained_variance   

In [248]:
FinalModel.learn(1.5 * 1e5)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 13.9     |
|    ep_rew_mean     | 1.14     |
| time/              |          |
|    fps             | 137      |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 2000     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.5        |
|    ep_rew_mean          | 1.02        |
| time/                   |             |
|    fps                  | 174         |
|    iterations           | 2           |
|    time_elapsed         | 22          |
|    total_timesteps      | 4000        |
| train/                  |             |
|    approx_kl            | 0.038805638 |
|    clip_fraction        | 0.176       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.415      |
|    explained_variance   | 0.504       |
|    learning_rate        | 0.

In [277]:
test_RL_models(FinalModel)

current data: data_easy
10.0 %, running acc: 76.25, task solved optimaly: 48.10126582278481 %, average steps to solve: 6.8354430379746836
20.0 %, running acc: 79.375, task solved optimaly: 50.943396226415096 %, average steps to solve: 6.018867924528302
30.0 %, running acc: 82.08333333333333, task solved optimaly: 53.97489539748954 %, average steps to solve: 6.2384937238493725
40.0 %, running acc: 81.875, task solved optimaly: 55.4858934169279 %, average steps to solve: 5.818181818181818
50.0 %, running acc: 82.5, task solved optimaly: 56.14035087719298 %, average steps to solve: 5.756892230576441
60.0 %, running acc: 83.54166666666667, task solved optimaly: 56.9937369519833 %, average steps to solve: 5.776617954070981
70.0 %, running acc: 83.75, task solved optimaly: 56.887298747763865 %, average steps to solve: 5.4973166368515205
80.0 %, running acc: 83.90625, task solved optimaly: 56.02503912363067 %, average steps to solve: 5.564945226917058
90.0 %, running acc: 83.61111111111111, t

In [17]:
testEnv = Gridworld(load_optimal= False)

In [211]:
curr = testEnv.reset()
testEnv.render()

[[['.' '.' '.' '.']
  ['.' '.' 'v' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '.']]

 [['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' 'v' 'O' '.']
  ['.' '.' '#' '.']]]


In [243]:
a = FinalModel.predict(curr, action_masks= testEnv.action_masks(), deterministic= False)[0]
curr, _, done, _ = testEnv.step(a)
print(testEnv.action_masks())
testEnv.render()
if done:
    print("finished")

[0. 1. 1. 0. 0. 0.]
[[['.' '.' '.' '^']
  ['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' '.' '#' '.']]

 [['.' '.' '.' '.']
  ['.' '.' '.' '.']
  ['.' 'v' 'O' '.']
  ['.' '.' '#' '.']]]
